<a href="https://colab.research.google.com/github/sherinjames-sj/GoogleColab_BigData/blob/main/RecommenderSystems_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install pyspark in google colab
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
!pip install findspark

import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
movies = spark.read.csv("movies.csv",header=True)
ratings = spark.read.csv("ratings.csv",header=True)

In [ ]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [ ]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
from pyspark.sql import functions as F

ratings = ratings.\
    withColumn('userId', F.col('userId').cast('integer')).\
    withColumn('movieId', F.col('movieId').cast('integer')).\
    withColumn('rating', F.col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [ ]:
# Sparsity shows how sparse is the matrix, e.g. 99% sparsity shows that, 99% of the entries in the matrix are missing.
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")


In [ ]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [ ]:
# Group data by movieId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|   2858|  204|
|     50|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [ ]:
# ALS MODEL --- Alternative Least Square Method 

# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
# Split the ratings dataframe into training and test data
(train, test) = ratings.randomSplit([0.8, 0.2], seed=42)

# Set the ALS hyperparameters
als = ALS(
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating", 
    rank =10, 
    maxIter =10, 
    regParam =.1,
    coldStartStrategy="drop", 
    nonnegative =True, 
    implicitPrefs = False
)

In [ ]:
# Fit the model to the training_data
als_model = als.fit(train)

# Generate predictions on the test_data
test_predictions = als_model.transform(test)
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|   4896|   4.0|  3.386292|
|   148|   5618|   3.0| 3.3454382|
|   148|   7153|   3.0| 3.5101054|
|   148|  40629|   5.0| 3.5732052|
|   148|  40815|   4.0|  3.416922|
|   148|  60069|   4.5| 3.6876173|
|   148|  68954|   4.0| 3.6699483|
|   148|  69844|   4.0| 3.6103592|
|   148|  79132|   1.5| 3.9365878|
|   148|  79702|   4.0| 3.6371486|
|   148|  81834|   4.0| 3.4486194|
|   148|  81847|   4.5| 3.4674408|
|   148|  98243|   4.5| 3.7436848|
|   148|  98491|   5.0|  3.231212|
|   148| 108932|   4.0| 3.6883512|
|   463|   1088|   3.5| 4.0561028|
|   463|   1221|   4.5| 4.1906967|
|   463|   2028|   4.5|  4.304896|
|   463|   2167|   3.0| 3.6635778|
|   463|   3448|   3.0| 3.6925979|
+------+-------+------+----------+
only showing top 20 rows



In [ ]:
# Import RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

# Complete the evaluator code
evaluator = RegressionEvaluator(metricName="rmse", labelCol="userId", predictionCol="rating")

# Extract the 3 parameters
print(evaluator.getMetricName())
print(evaluator.getLabelCol())
print(evaluator.getPredictionCol())


# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print ('RMSE = ', RMSE)

rmse
userId
rating
RMSE =  368.0588125025974


In [ ]:
# Generate n Recommendations for all users
nrecommendations = als_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{25771, 6.082720...|
|     3|[{4442, 5.1741395...|
|     5|[{25771, 5.208932...|
|     6|[{6732, 5.1475816...|
|     9|[{104875, 5.33092...|
|    12|[{7121, 5.936296}...|
|    13|[{5075, 5.476552}...|
|    15|[{89904, 4.928991...|
|    16|[{25771, 4.727194...|
|    17|[{33649, 5.084311...|
+------+--------------------+



In [ ]:
from pyspark.sql.functions import split, explode

nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', F.col("rec_exp.movieId"), F.col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|  25771|6.0827203|
|     1| 177593|5.8191457|
|     1|  89904|5.7939825|
|     1|  59814| 5.646638|
|     1|  33649| 5.577319|
|     1| 104875|5.5757227|
|     1| 188751|5.5581303|
|     1|  58301| 5.549706|
|     1|   6666|5.5312166|
|     1| 117531|5.5190425|
+------+-------+---------+



In [ ]:
#Explaining the recommendations 
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|   7121|   100|5.2964478|   Adam's Rib (1949)|      Comedy|Romance|
|   6732|   100| 5.207034|Hello, Dolly! (1969)|Comedy|Musical|Ro...|
|  32892|   100| 5.045046|Ivan's Childhood ...|           Drama|War|
|   8235|   100| 5.021771| Safety Last! (1923)|Action|Comedy|Rom...|
|   6201|   100| 5.021771|    Lady Jane (1986)|       Drama|Romance|
| 177593|   100| 4.987484|Three Billboards ...|         Crime|Drama|
|  74282|   100| 4.938864|Anne of Green Gab...|Children|Drama|Ro...|
|  67618|   100| 4.932314|Strictly Sexual (...|Comedy|Drama|Romance|
| 184245|   100|4.9018416|De platte jungle ...|         Documentary|
|  84273|   100|4.9018416|Zeitgeist: Moving...|         Documentary|
+-------+------+---------+--------------------+--------------------+



In [ ]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

